In [1]:
!pip3 install nltk
!pip3 install numpy
!pip3 install Pillow
!pip3 install pycocotools

In [2]:
import nltk
import pickle
import os
import numpy as np
from PIL import Image
from collections import Counter
from pycocotools.coco import COCO
import glob
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
import torch.utils.data as data

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yuhei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
!mkdir data
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip -P ./data/
!wget http://images.cocodataset.org/zips/train2014.zip -P ./data/

!unzip ./data/annotations_trainval2014.zip -d ./data/
!rm ./data/annotations_trainval2014.zip
!unzip ./data/train2014.zip -d ./data/
!rm ./data/train2014.zip

�T�u�f�B���N�g���܂��̓t�@�C�� data �͊��ɑ��݂��܂��B
'wget' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
'wget' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
'unzip' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
'rm' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
'unzip' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
'rm' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


In [4]:
class Vocabulary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx: #重複がない単語、idを追加
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx: #未知の単語である場合、'<unk>'のidを返す
            return self.word2idx['<unk>']
        return self.word2idx[word] #単語と対応するidを返す

    def __len__(self):
        return len(self.word2idx) #単語の数を返す

def make_vocab(json, threshold):
    coco = COCO(json)
    counter = Counter()
    ids = coco.anns.keys() #各データの注釈のリストを返す
    for i, id in enumerate(ids):
        caption = str(coco.anns[id]['caption']) #各データのキャプションを返す
        tokens = nltk.tokenize.word_tokenize(caption.lower()) #キャプションを小文字に変換し、単語ごとに区分する
        counter.update(tokens) #これまで出現した各単語の個数を更新する

        if (i+1) % 1000 == 0:
            print("[{}/{}] Tokenized captions.".format(i+1, len(ids)))

    words = [word for word, cnt in counter.items() if cnt >= threshold]

    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    for i, word in enumerate(words):
        vocab.add_word(word)
    return vocab

vocab = make_vocab(json='./data/annotations/captions_val2014.json', threshold=4)
vocab_path = './data/vocab.pkl'
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab, f)
print("Total vocabulary size: {}".format(len(vocab)))
print("Saved vocabulary wrapper to '{}'".format(vocab_path))



loading annotations into memory...
Done (t=0.65s)
creating index...
index created!
[1000/202654] Tokenized captions.
[2000/202654] Tokenized captions.
[3000/202654] Tokenized captions.
[4000/202654] Tokenized captions.
[5000/202654] Tokenized captions.
[6000/202654] Tokenized captions.
[7000/202654] Tokenized captions.
[8000/202654] Tokenized captions.
[9000/202654] Tokenized captions.
[10000/202654] Tokenized captions.
[11000/202654] Tokenized captions.
[12000/202654] Tokenized captions.
[13000/202654] Tokenized captions.
[14000/202654] Tokenized captions.
[15000/202654] Tokenized captions.
[16000/202654] Tokenized captions.
[17000/202654] Tokenized captions.
[18000/202654] Tokenized captions.
[19000/202654] Tokenized captions.
[20000/202654] Tokenized captions.
[21000/202654] Tokenized captions.
[22000/202654] Tokenized captions.
[23000/202654] Tokenized captions.
[24000/202654] Tokenized captions.
[25000/202654] Tokenized captions.
[26000/202654] Tokenized captions.
[27000/202654] T

In [5]:
def resize_image(image, size):
  return image.resize((size,size), Image.ANTIALIAS) #ジャギーを修正しリサイズ

def resize_images(image_dir, output_dir, size):
  if not os.path.exists(output_dir):#保存先のディレクトリが存在しない場合、新たに作成
    os.makedirs(output_dir)

  images = os.listdir(image_dir) #ディレクトリの全ての画像のファイル名をリストで返す
  num_images = len(images) #画像の個数を返す

  for i, image in enumerate(images):
    with open(os.path.join(image_dir, image), "r+b") as f: #各画像のファイルを取得
      with Image.open(f) as img: #各画像をPillow形式で開く
        img = resize_image(img, size)
        img.save(os.path.join(output_dir, image), img.format) #元の形式でリサイズした画像を保存
      if (i+1)%100 == 0:
        print ("[{}/{}] Resized the images and saved into '{}'."
                   .format(i+1, num_images, output_dir))

image_dir = "./data/train2014/"
output_dir = "./data/resized2014/"
image_size = 256
resize_images(image_dir, output_dir, image_size)



<ipython-input-5-70403d7e7f63>:2: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  return image.resize((size,size), Image.ANTIALIAS) #ジャギーを修正しリサイズ


[100/82783] Resized the images and saved into './data/resized2014/'.
[200/82783] Resized the images and saved into './data/resized2014/'.
[300/82783] Resized the images and saved into './data/resized2014/'.
[400/82783] Resized the images and saved into './data/resized2014/'.
[500/82783] Resized the images and saved into './data/resized2014/'.
[600/82783] Resized the images and saved into './data/resized2014/'.
[700/82783] Resized the images and saved into './data/resized2014/'.
[800/82783] Resized the images and saved into './data/resized2014/'.
[900/82783] Resized the images and saved into './data/resized2014/'.
[1000/82783] Resized the images and saved into './data/resized2014/'.
[1100/82783] Resized the images and saved into './data/resized2014/'.
[1200/82783] Resized the images and saved into './data/resized2014/'.
[1300/82783] Resized the images and saved into './data/resized2014/'.
[1400/82783] Resized the images and saved into './data/resized2014/'.
[1500/82783] Resized the imag

In [6]:
class CocoDataset(data.Dataset):
  def __init__(self, root, json, vocab, transform=None):
    self.root = root #現在のディレクトリ
    self.coco = COCO(json)
    self.ids = list(self.coco.anns.keys()) #各データの注釈をリスト化
    self.vocab = vocab
    self.transform = transform #追加の前処理を定義

  def __getitem__(self, index):
    coco = self.coco
    vocab = self.vocab
    ann_id = self.ids[index] #指定したindex番目のデータidを返す
    caption = self.coco.anns[ann_id]["caption"] #指定したデータのキャプションを返す
    img_id = self.coco.anns[ann_id]["image_id"] #指定したデータの画像idを返す
    path = coco.loadImgs(img_id)[0]["file_name"] #指定したデータの画像データにおけるファイル名を取得

    image = Image.open(os.path.join(self.root, path)).convert("RGB")#指定したデータの画像を開き、RGBに変換
    if self.transform is not None: #追加の前処理がある場合は実行
      image = self.transform(image)

    tokens = nltk.tokenize.word_tokenize(str(caption).lower()) #キャプションを小文字に変換し、単語ごとに分割する
    caption = []
    caption.append(vocab('<start>'))
    caption.extend([vocab(token) for token in tokens]) #Vocabularyの__call__を用いて、単語のidをキャプションに出現する順に格納
    caption.append(vocab('<end>'))
    target = torch.Tensor(caption)

    return image, target

  def __len__(self):
    return len(self.ids)

In [7]:
def collate_fn(data):
  data.sort(key=lambda x: len(x[1]), reverse = True) #キャプションの長さを基準に、降順にソートする
  images, captions = zip(*data) #画像とキャプションを分割する

  images = torch.stack(images, 0) #画像を0次元方向に結合
  lengths = [len(cap) for cap in captions] #キャプションの長さを取得
  targets = torch.zeros(len(captions), max(lengths)).long() #行方向に全キャプションの数、列方向に最大長のキャプションの長さとなる2次元テンソルを宣言

  for i, cap in enumerate(captions):
    end = lengths[i] #各キャプションの長さを取得
    targets[i, :end] = cap[:end] #各キャプションの長さまで代入し、end以降は0となるtensorを作成

  return images, targets, lengths

def get_loader(root, json, vocab, transform, batch_size, shuffle, num_workers):
  coco = CocoDataset(root = root,
                     json = json,
                     vocab = vocab,
                     transform = transform)

  data_loader = torch.utils.data.DataLoader(dataset = coco,
                                            batch_size = batch_size,
                                            shuffle = shuffle,
                                            num_workers = num_workers,
                                            collate_fn = collate_fn)

  return data_loader



In [8]:
class Encoder(nn.Module):
  def __init__(self, embed_size):
    super().__init__()
    resnet = models.resnet152(pretrained = True) #学習済みresnetモデル
    modules = list(resnet.children())[:-1] #最終層を除くresnetをリスト化
    self.resnet = nn.Sequential(*modules)
    self.linear = nn.Linear(resnet.fc.in_features, embed_size) #resnetの最終層を設定
    self.bn = nn.BatchNorm1d(embed_size, momentum=0.01) #1次元のバッチ正規化層を設定

  def forward(self, images): #順伝播
    with torch.no_grad(): #最終層まで学習済みであるため、勾配更新を停止
      features = self.resnet(images) #画像データを入力
    features = features.reshape(features.size(0), -1) #(バッチサイズ, features.shape(1) * features.shape(2))にリサイズ
    features = self.bn(self.linear(features)) #線形層に入力後、1次元バッチ正規化に入力
    return features


In [11]:
class Decoder(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=20):
    super().__init__()
    self.embed = nn.Embedding(vocab_size, embed_size)#埋め込み層を設定
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)#LSTM層を設定
    self.linear = nn.Linear(hidden_size, vocab_size)#線形層を設定（入力の次元と合わせる）
    self.max_seg_length = max_seq_length

  def forward(self, features, captions, lengths):
    embeddings = self.embed(captions)
    embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)#Encoderの出力と埋め込み層の出力を結合
    packed = pack_padded_sequence(embeddings, lengths, batch_first=True)#可変系列のためパディング
    hiddens, _ = self.lstm(packed)
    outputs = self.linear(hiddens[0])

    return outputs

  def predict(self, features, states=None):
    predicted_ids = []#予測した単語のidを格納するリスト
    inputs = features.unsqueeze(1)

    for i in range(self.max_seg_length):
      hiddens, states = self.lstm(inputs, states)#heiddens:LSTMの出力、states:隠れ層の記憶状態
      outputs = self.linear(hiddens.squeeze(1))#線形層に対応して、次元数を縮小する
      _, predicted = outputs.max(1)#最大値を_に、その位置をpredictedに格納
      predicted_ids.append(predicted)#予測したidを格納
      inputs = self.embed(predicted)#predictを再度embedに入力する
      inputs = inputs.unsqueeze(1)#再度次元数を拡張

    predicted_ids = torch.stack(predicted_ids, 1)#predicted_idsを1次元方向にスタック結合

    return predicted_ids







In [ ]:
device = torch.device("cuda")#GPUを使用、不可能な場合はcpuを用いる

model_path = 'models/'
crop_size = 224#画像の切り取りサイズ
vocab_path = 'data/vocab.pkl'
image_dir ='data/resized2014'
caption_path='data/annotations/captions_train2014.json'
log_step=10#ログの出力間隔
save_step=1000#モデルを保存する間隔（バッチ回数）
embed_size=256#埋め込みベクトルの次元数
hidden_size=512#LSTMの隠れ層の次元数
num_layers=1#LSTMの隠れ層の層数
num_epochs=10#エポック数
batch_size=128#バッチサイズ
num_workers=2#並行処理の数
learning_rate=0.001#学習率

os.makedirs(model_path, exist_ok=True)

transform = transforms.Compose([
    transforms.RandomCrop(crop_size),#ランダムに画像を切り取り（crop_size:切り取り後のサイズ）
    transforms.RandomHorizontalFlip(),#画像を水平方向に反転
    transforms.ToTensor(),#テンソル化((W, H, C)→(C, W, H)に変換、データの範囲[0,255]を[0,1.0]に変換)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])#チャンネルごとに標準化
])

with open(vocab_path, "rb") as f:
  vocab = pickle.load(f)

data_loader = get_loader(image_dir, caption_path, vocab, transform, batch_size,
                          shuffle=True, num_workers=num_workers)

encoder = Encoder(embed_size).to(device)
decoder = Decoder(embed_size, hidden_size, len(vocab), num_layers).to(device)

criterion = nn.CrossEntropyLoss()#多値分類であるためクロスエントロピー誤差を設定
params = list(decoder.parameters()) + list(encoder.linear.parameters()) + list(encoder.bn.parameters())#decoderとencoderの最適化パラメータを集めてparamsに設定

optimizer = torch.optim.Adam(params, lr=learning_rate)

total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, captions, lengths) in enumerate(data_loader):

        images = images.to(device)
        captions = captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]#captionsとlengthsでpack_padded_sequenceを作成

        features = encoder(images)
        outputs = decoder(features, captions, lengths)
        loss = criterion(outputs, targets)
        decoder.zero_grad()
        encoder.zero_grad()
        loss.backward()
        optimizer.step()

        if i % log_step == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Perplexity: {:5.4f}'
                  .format(epoch, num_epochs, i, total_step, loss.item(), np.exp(loss.item())))

        if (i+1) % save_step == 0:
            torch.save(decoder.state_dict(), os.path.join(
                model_path, 'decoder-{}-{}.ckpt'.format(epoch+1, i+1)))
            torch.save(encoder.state_dict(), os.path.join(
                model_path, 'encoder-{}-{}.ckpt'.format(epoch+1, i+1)))

loading annotations into memory...
Done (t=1.29s)
creating index...
index created!


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [0/10], Step [0/3236], Loss: 8.8971, Perplexity: 7310.8634
Epoch [0/10], Step [10/3236], Loss: 5.7531, Perplexity: 315.1618
Epoch [0/10], Step [20/3236], Loss: 5.2506, Perplexity: 190.6742
Epoch [0/10], Step [30/3236], Loss: 4.9861, Perplexity: 146.3583
Epoch [0/10], Step [40/3236], Loss: 4.4236, Perplexity: 83.3983
Epoch [0/10], Step [50/3236], Loss: 4.3541, Perplexity: 77.7953
Epoch [0/10], Step [60/3236], Loss: 4.1049, Perplexity: 60.6351
Epoch [0/10], Step [70/3236], Loss: 3.9111, Perplexity: 49.9545
Epoch [0/10], Step [80/3236], Loss: 3.7270, Perplexity: 41.5537
Epoch [0/10], Step [90/3236], Loss: 3.7873, Perplexity: 44.1389
Epoch [0/10], Step [100/3236], Loss: 3.7898, Perplexity: 44.2485
Epoch [0/10], Step [110/3236], Loss: 3.6659, Perplexity: 39.0899
Epoch [0/10], Step [120/3236], Loss: 3.5358, Perplexity: 34.3210
Epoch [0/10], Step [130/3236], Loss: 3.6237, Perplexity: 37.4765
Epoch [0/10], Step [140/3236], Loss: 3.6152, Perplexity: 37.1594
Epoch [0/10], Step [150/3236], 

In [ ]:
!wget https://www.dropbox.com/s/ne0ixz5d58ccbbz/pretrained_model.zip
!unzip pretrained_model.zip
!mv encoder-5-3000.pkl decoder-5-3000.pkl models
!wget https://www.dropbox.com/s/26adb7y9m98uisa/vocap.zip
!unzip vocap.zip
!mv vocab.pkl models

In [ ]:
!pip uninstall googletrans
!pip install googletrans==4.0.0-rc1

In [ ]:
!wget 'https://tensorflow.org/images/surf.jpg'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_image(image_path, transform=None):
    image = Image.open(image_path).convert('RGB')
    image = image.resize([224, 224], Image.LANCZOS)

    if transform is not None:
        image = transform(image).unsqueeze(0)

    return image

encoder_path ='models/encoder-5-3000.pkl'
decoder_path ='models/decoder-5-3000.pkl'
vocab_path ='models/vocab.pkl'

embed_size=256
hidden_size=512
num_layers=1

image_path = './surf.jpg'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)

encoder = Encoder(embed_size).eval()#評価モードに変更
decoder = Decoder(embed_size, hidden_size, len(vocab), num_layers)
encoder = encoder.to(device)
decoder = decoder.to(device)

encoder.load_state_dict(torch.load(encoder_path))#Encoderに学習後の重みをロード
decoder.load_state_dict(torch.load(decoder_path))#Decoderに学習後の重みをロード

image = load_image(image_path, transform)
image_tensor = image.to(device)

feature = encoder(image_tensor)
predicted_ids = decoder.predict(feature)
predicted_ids = predicted_ids[0].cpu().numpy()

predicted_caption = []
for word_id in predicted_ids:
    word = vocab.idx2word[word_id]#予測したidに対応する単語を取得
    predicted_caption.append(word)
    if word == '<end>':
        break
sentence = ' '.join(predicted_caption[1:-2])

from googletrans import Translator
translator = Translator()
translation = translator.translate(sentence, dest='ja')

image = Image.open(image_path)
plt.imshow(np.asarray(image))
plt.show()
print (sentence)
print(translation.text)